In [2]:
def wrangle(path):
    dataframe = (pd.read_csv(path).
             loc[data["date"] < '2021-07-13'].
             drop(columns = ['playoff', 'neutral', 'season', 'date']).
             reset_index().
             drop(columns = ['index'])
)
    return dataframe